### Configure Ollama with structured outputs ###

In [1]:
import os
import pandas as pd
import numpy as np
import logging
import time
from pathlib import Path
from pydantic import BaseModel

logger = logging.getLogger(__name__)

import ollama
from ollama import Client

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
from llmt.llmtools import Prompt
from llmt.llmtools import process_prompt
from llmt.openai import OpenAIModel, OpenAI
from llmt.openai import MentalHealth, OutpatientServices, InpatientServices, create_messages
from llmt.performance import Performance
# print(llmt.__version__)

### Custom Ollama Client ###

In [2]:
client = Client(
  host='http://ollama:11434',
  headers={'x-some-header': 'some-value'}
)

In [3]:
model = 'llama2:7b'
client.pull(model)
ollama_models = client.list().get('models')
ollama_model_list = [model['model'] for model in ollama_models]
print(ollama_model_list)

['llama2:7b']


In [4]:
response = client.chat(model=model, messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response)

model='llama2:7b' created_at='2025-05-15T10:39:55.877211775Z' done=True done_reason='stop' total_duration=10568306692 load_duration=907580128 prompt_eval_count=26 prompt_eval_duration=222311995 eval_count=335 eval_duration=9436415952 message=Message(role='assistant', content="\nThe sky appears blue because of a phenomenon called Rayleigh scattering, which occurs when sunlight travels through the Earth's atmosphere. The atmosphere contains small particles such as molecules of gases like nitrogen and oxygen, as well as tiny dust particles. When sunlight enters the atmosphere, it encounters these particles and is scattered in all directions.\n\nThe shortest wavelengths of light, such as violet and blue, are scattered more than longer wavelengths, such as red and orange. This is known as Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described the phenomenon in the late 19th century.\n\nAs a result of this scattering, the blue light is distributed throughou

### Structured Outputs ###

In [5]:
class Country(BaseModel):
  name: str
  capital: str
  languages: list[str]

In [6]:
response = client.chat(messages=[{'role': 'user',
                                  'content': 'Tell me about Canada.'}],
                       model=model,
                       format=Country.model_json_schema())

In [7]:
country = Country.model_validate_json(response.message.content)
print(country)

name='Canada' capital='Ottawa' languages=['English', 'French', 'Indigenous languages']
